# Step 1: Geo-Level Scaled Media Simulation Validation

This notebook validates the **corrected** first step of the revamped FastResponseCurves implementation:
**Geo-level scaled media simulation from 0 to 2x maximum historical scaled media per geography.**

## Key Correction
- **Previous**: Simulated spend scenarios (wrong - values in millions)
- **Current**: Simulates scaled media scenarios (correct - values 0-15 range)
- **Data source**: `mmm.media_tensors.media_scaled` (the data actually used in transformations)

## Objectives
1. Test the `GeoMediaSimulator` class with real model data
2. Validate scaled media range generation per geo and channel
3. Visualize scaled media distributions and ranges
4. Confirm compatibility with adstock/hill transformations
5. Confirm readiness for Step 2 (media transformations)

## Setup and Imports

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
from pathlib import Path
import sys
import tensorflow as tf

# Add current directory to path to import our module
sys.path.append('.')
from geo_media_simulation import GeoMediaSimulator
from meridian.model import model

# Set up plotting
plt.style.use('default')
# sns.set_palette('husl')
%matplotlib inline

print("📦 Imports completed successfully!")

📦 Imports completed successfully!


## Load Test Model (ALDI)

In [21]:
# Model path
MODEL_PATH = "/Users/mariappan.subramanian/Library/CloudStorage/OneDrive-TheTradeDesk/MMM/Media Parameter Analysis/Dev/MMMFeasibility/model_objects/0_test_working_spend_ALDI_US_Starcom.pkl"

print(f"📁 Loading ALDI model...")
print(f"   Path: {Path(MODEL_PATH).name}")

# Load model
mmm = model.load_mmm(MODEL_PATH)

# Display model info
print(f"\n📊 Model Information:")
print(f"   Geos: {mmm.n_geos}")
print(f"   Time periods: {mmm.n_times}")
print(f"   Media channels: {mmm.n_media_channels}")
print(f"   R&F channels: {mmm.n_rf_channels}")

if mmm.input_data.media_channel is not None:
    print(f"   Media channel names: {list(mmm.input_data.media_channel.values)}")
if mmm.input_data.geo is not None:
    print(f"   First 5 geos: {list(mmm.input_data.geo.values[:5])}")
if mmm.input_data.time is not None:
    print(f"   Time range: {mmm.input_data.time.values[0]} to {mmm.input_data.time.values[-1]}")

📁 Loading ALDI model...
   Path: 0_test_working_spend_ALDI_US_Starcom.pkl

📊 Model Information:
   Geos: 28
   Time periods: 78
   Media channels: 3
   R&F channels: 0
   Media channel names: ['TV', 'Display', 'Video']
   First 5 geos: [np.str_('ALABAMA'), np.str_('ARIZONA'), np.str_('CALIFORNIA'), np.str_('CONNECTICUT'), np.str_('FLORIDA')]
   Time range: 2024-01-06 to 2025-06-28


## Initialize GeoMediaSimulator

In [30]:
print("⚡ Initializing GeoMediaSimulator...")

# Initialize simulator
simulator = GeoMediaSimulator(mmm)

# print(f"\n✅ Simulator initialized successfully!")
# print(f"   Historical scaled media shape: {simulator.historical_scaled_media.shape}")
# print(f"   Channel names: {simulator.channel_names}")
# print(f"   Number of geos: {simulator.n_geos}")
# print(f"   Number of time periods: {simulator.n_times}")
# print(f"   Number of channels: {simulator.n_channels}")
# print(f"   Value range: [{float(simulator.historical_scaled_media.numpy().min()):.3f}, {float(simulator.historical_scaled_media.numpy().max()):.3f}]")

⚡ Initializing GeoMediaSimulator...
📊 Scaled media data extracted:
   Shape: (28, 78, 3) (geos, times, channels)
   Channels: ['TV', 'Display', 'Video']
   Value range: [[0.09167854 0.04971217 0.05246428], [ 7.536399  5.281188 13.257745]]


## Generate Geo-Level Scaled Media Scenarios

In [31]:
print("🗺️ Generating geo-level scaled media scenarios...")

# Generate scaled media scenarios
media_scenarios, metadata = simulator.generate_geo_media_scenarios(
    max_multiplier=2.0,  # 0 to 200% of historical max
    num_steps=50,        # 50 steps per geo
    selected_geos=None,  # All geos
    selected_times=None  # All time periods
)

print(f"\n📊 Generation Results:")
print(f"   Media scenarios shape: {media_scenarios.shape}")
print(f"   Expected shape: ({simulator.n_geos}, 50, {simulator.n_channels})")
print(f"   Max multiplier: {metadata['max_multiplier']}x")
print(f"   Steps per geo: {metadata['num_steps']}")
print(f"   Channels: {metadata['channel_names']}")
print(f"   Data type: {metadata['data_type']}")
print(f"   Value range: [{metadata['value_range'][0]:.3f}, {metadata['value_range'][1]:.3f}]")
print(f"   Simulation range: [{metadata['simulation_range'][0]:.3f}, {metadata['simulation_range'][1]:.3f}]")

# Validate shapes
expected_shape = (simulator.n_geos, 50, simulator.n_channels)
actual_shape = media_scenarios.shape
print(f"\n✅ Shape validation: {'PASS' if actual_shape == expected_shape else 'FAIL'}")

🗺️ Generating geo-level scaled media scenarios...
🗺️ Generating geo-level scaled media scenarios...
   Max multiplier: 2.0x
   Steps per geo: 50
   Historical scaled media shape: (28, 78, 3)
   Geo max scaled media shape: (28, 3)
   Scaled media range: [1.336, 13.258]
✅ Geo scaled media scenarios generated:
   Output shape: (28, 50, 3) (geos, steps, channels)
   Scaled media range per geo: 0 to 2.0x historical max
   Max simulation values: ['15.073', '10.562', '26.515']

📊 Generation Results:
   Media scenarios shape: (28, 50, 3)
   Expected shape: (28, 50, 3)
   Max multiplier: 2.0x
   Steps per geo: 50
   Channels: ['TV', 'Display', 'Video']
   Data type: scaled_media
   Value range: [1.336, 13.258]
   Simulation range: [0.000, 26.515]

✅ Shape validation: PASS
